<a href="https://colab.research.google.com/github/damianwgriggs/Quantum-Notary-/blob/main/Quantum_Security_Thing_Notary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install cirq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.8/670.8 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.5/430.5 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 69.7 MB/s eta 0:00:00


In [3]:
# ----------------------------------------------------------------------
# SIMULATION: THE "QUANTUM NOTARY" SMART CONTRACT PROTOCOL (V2)
# ----------------------------------------------------------------------
# OBJECTIVE: Verify the logical flow of the four "Lego blocks"
# 1. Drafting (Time Rollback)
# 2. Agreement (Entanglement)
# 3. Execution (Teleportation)
# 4. Finalization (Schrödinger's Server)
# ----------------------------------------------------------------------

import sys
import subprocess

# --- STEP 0: AUTO-INSTALLER ---
# This block checks if Qiskit is installed. If not, it installs it automatically.
def install(package):
    print(f"Installing {package}...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    import qiskit
    from qiskit import QuantumCircuit, transpile
    from qiskit_aer import AerSimulator
    import matplotlib.pyplot as plt
    print("✅ Qiskit is already installed.")
except ImportError:
    print("⚠️ Qiskit not found. Installing dependencies now (this may take a minute)...")
    install("qiskit")
    install("qiskit-aer")
    install("matplotlib")
    install("pylatexenc") # For better circuit visualization

    # Re-import after installation
    import qiskit
    from qiskit import QuantumCircuit, transpile
    from qiskit_aer import AerSimulator
    import matplotlib.pyplot as plt
    print("✅ Installation complete.")

from math import pi

# Initialize the simulator
simulator = AerSimulator()
print(f"--- 🔬 QUANTUM NOTARY SIMULATION ONLINE: {simulator.name} ---")


# ----------------------------------------------------------------------
# PHASE 1: The "Drafting Phase" (Time Rollback / Undo Functionality)
# ----------------------------------------------------------------------
print("\n" + "="*60)
print("PHASE 1: DRAFTING (TIME ROLLBACK)")
print("Logic: Apply 'Draft Clause' -> Apply 'Inverse' -> Result must be |0>.")
print("="*60)

qc_draft = QuantumCircuit(1, 1)

# 1. Forward Operation (The "Draft Clause")
qc_draft.rx(pi/2, 0)  # Arbitrary complex rotation
qc_draft.rz(pi/4, 0)

# 2. Apply Time Rollback (The .inverse() method)
qc_draft.compose(qc_draft.inverse(), inplace=True)

# 3. Proof: Measure
qc_draft.measure(0, 0)

# Execute
job = simulator.run(transpile(qc_draft, simulator), shots=1000)
counts = job.result().get_counts()

print(f"Drafting Check (Should be {{'0': 1000}}): {counts}")
if counts.get('0') == 1000:
    print("✅ SUCCESS: Draft successfully rolled back to zero state.")
else:
    print("❌ FAILURE: Draft was not perfectly undone.")


# ----------------------------------------------------------------------
# PHASE 2: The "Agreement Phase" (Entanglement)
# ----------------------------------------------------------------------
print("\n" + "="*60)
print("PHASE 2: AGREEMENT (ENTANGLEMENT)")
print("Logic: Create Bell Pair -> Measure. Bank and Client must match.")
print("="*60)

qc_agree = QuantumCircuit(2, 2)

# 1. Entangle (H + CNOT)
qc_agree.h(0)
qc_agree.cx(0, 1)

# 2. Measure both
qc_agree.measure([0, 1], [0, 1])

# Execute
job = simulator.run(transpile(qc_agree, simulator), shots=1000)
counts = job.result().get_counts()

print(f"Agreement Measurement: {counts}")
# We expect roughly 50% '00' and 50% '11', and 0% '01' or '10'
if '01' not in counts and '10' not in counts:
    print("✅ SUCCESS: Perfect correlation. No disagreement states found.")
else:
    print("❌ FAILURE: Correlation broken.")


# ----------------------------------------------------------------------
# PHASE 3: The "Execution Phase" (Teleportation / Super Cut)
# ----------------------------------------------------------------------
print("\n" + "="*60)
print("PHASE 3: EXECUTION (TELEPORTATION)")
print("Logic: Bank sends Key (|1>) -> Source Destroyed -> Notary Receives |1>.")
print("="*60)

# q0=Message, q1=Ancilla, q2=Notary
qc_tele = QuantumCircuit(3, 3)

# 1. Prepare the Key (The "YES" vote, state |1>)
qc_tele.x(0)

# 2. Create Entanglement (Bank Ancilla <-> Notary)
qc_tele.h(1)
qc_tele.cx(1, 2)

# 3. Bank measures their own qubits (Destructive step)
qc_tele.cx(0, 1)
qc_tele.h(0)
qc_tele.measure(0, 0) # Measure Message
qc_tele.measure(1, 1) # Measure Ancilla

# 4. Notary applies corrections based on Bank's message
with qc_tele.if_test((0, 1)): qc_tele.z(2) # If bit 0 is 1, Z gate
with qc_tele.if_test((1, 1)): qc_tele.x(2) # If bit 1 is 1, X gate

# 5. Verify Notary has the key
qc_tele.measure(2, 2)

# Execute
job = simulator.run(transpile(qc_tele, simulator), shots=1000)
counts = job.result().get_counts()

print(f"Teleportation Results (Format: 'NotaryBit BankBit BankBit'): {counts}")

# Filter to check if the Notary bit (the left-most bit in string, or right-most qubit) is '1'
# Qiskit bitstrings are read right-to-left (Q2 Q1 Q0).
# We want Q2 (Notary) to be 1. So keys should look like '100', '101', '110', '111'.
success_count = sum(v for k, v in counts.items() if k.startswith('1'))
print(f"Packets where Notary received Key (|1>): {success_count}/1000")

if success_count == 1000:
    print("✅ SUCCESS: Key successfully teleported to Notary.")
else:
    print("❌ FAILURE: Key did not arrive reliably.")


# ----------------------------------------------------------------------
# PHASE 4: The "Finalization Phase" (Schrödinger's Server)
# ----------------------------------------------------------------------
print("\n" + "="*60)
print("PHASE 4: FINALIZATION (TOFFOLI GATE)")
print("Logic: Server generates contract (State 1) ONLY if Bank AND Client send keys.")
print("="*60)

def run_server_check(bank_sent, client_sent):
    qc_server = QuantumCircuit(3, 1)

    # Inputs
    if bank_sent: qc_server.x(0)
    if client_sent: qc_server.x(1)

    # The Logic: Toffoli (CCX)
    # Control 0 (Bank), Control 1 (Client) -> Target 2 (Server State)
    qc_server.ccx(0, 1, 2)

    qc_server.measure(2, 0)

    job = simulator.run(transpile(qc_server, simulator), shots=1)
    result = list(job.result().get_counts().keys())[0]
    return int(result)

# Test Scenario A: Only Bank sends key
res_a = run_server_check(bank_sent=True, client_sent=False)
print(f"Scenario A (Bank=YES, Client=NO): Server State = {res_a} " + ("(Contract Pending)" if res_a == 0 else "(ERROR)"))

# Test Scenario B: Both send keys
res_b = run_server_check(bank_sent=True, client_sent=True)
print(f"Scenario B (Bank=YES, Client=YES): Server State = {res_b} " + ("(Contract Generated!)" if res_b == 1 else "(ERROR)"))

if res_a == 0 and res_b == 1:
    print("✅ SUCCESS: Schrödinger's Server logic verified.")
else:
    print("❌ FAILURE: Server logic incorrect.")

print("\n--- 🏁 QUANTUM NOTARY SIMULATION COMPLETE ---")

⚠️ Qiskit not found. Installing dependencies now (this may take a minute)...
Installing qiskit...
Installing qiskit-aer...
Installing matplotlib...
Installing pylatexenc...
✅ Installation complete.
--- 🔬 QUANTUM NOTARY SIMULATION ONLINE: aer_simulator ---

PHASE 1: DRAFTING (TIME ROLLBACK)
Logic: Apply 'Draft Clause' -> Apply 'Inverse' -> Result must be |0>.
Drafting Check (Should be {'0': 1000}): {'0': 1000}
✅ SUCCESS: Draft successfully rolled back to zero state.

PHASE 2: AGREEMENT (ENTANGLEMENT)
Logic: Create Bell Pair -> Measure. Bank and Client must match.
Agreement Measurement: {'00': 530, '11': 470}
✅ SUCCESS: Perfect correlation. No disagreement states found.

PHASE 3: EXECUTION (TELEPORTATION)
Logic: Bank sends Key (|1>) -> Source Destroyed -> Notary Receives |1>.
Teleportation Results (Format: 'NotaryBit BankBit BankBit'): {'111': 256, '110': 239, '101': 247, '100': 258}
Packets where Notary received Key (|1>): 1000/1000
✅ SUCCESS: Key successfully teleported to Notary.

PHA